In [29]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.llms import cohere
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Cohere
from langchain_cohere import ChatCohere
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
import chromadb
import cohere


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")
openai_api_key=os.getenv("OPENAI_API_KEY")

In [15]:

CHROMA_PATH=r"chroma_db"
chroma_client=chromadb.PersistentClient(path=CHROMA_PATH)
collection=chroma_client.get_or_create_collection(name="LLm")

loader=PyPDFLoader("C:\\Users\\SAUNAK MITRA\\OneDrive\\Desktop\\jupyter\\assignment\\data\\NIPS-2017-attention-is-all-you-need-Paper.pdf")
raw_documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len
)

chunks=text_splitter.split_documents(raw_documents)
documents=[]
metadata=[]
ids=[]

i= 0
for chunk in chunks:
    documents.append(chunk.page_content)
    ids.append("ID"+str(i))
    metadata.append(chunk.metadata)
    i+=1
collection.upsert(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

C:\Users\SAUNAK MITRA\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:34<00:00, 2.43MiB/s]


In [21]:
pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from openai import OpenAI

In [39]:
user_query=input("what is the data about?\n\n")

results=collection.query(query_texts=[user_query],n_results=1)

client=OpenAI(api_key=openai_api_key)

system_prompt="""
You are a helpful assistant. You answer questions about llm.
But you only answer based on knowledge I'm providing you.you don't use your internal knowledge and you don't make things up.

If you don't know the answer, just say: I don't know.

---------------------
The data: 
"""+str(results['documents'])+"""
"""+str(results['metadatas'])+"""
"""

response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content":user_query}
    ]
)
print("\n\n------------------\n\n")
print(response.choices[0].message.content)

SyntaxError: invalid syntax (2070960518.py, line 5)